In [1]:
import re
import pandas as pd

In [2]:
class ExtractDataFrame:
    
    '''
    This module will help parsing the whatsapp chats data.
        Parameters:
            File_path (string): The path of the chats files
        Functions:
            load_file(object) -> File pointer
            is_newEntry(object, string) -> Boolean
    '''
    
    def __init__(self, file_path):
        self.path = file_path
        self.data = []
        
    def load_file(self):
        '''
        This function loads the chat file
        '''
        file = open(self.path, 'r', encoding='utf-8')
        return file
    
    def is_newEntry(self, line: str) -> bool:
        '''
        This function returns if the line is a new message or continuation of the previous one
        '''
        date_time = '([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -'
        test = re.match(date_time, line)
        if test is not None:
            return True
        else:
            return False
    
    def display(self):
        f = self.load_file()
        while True:
            f.readline()
            line = f.readline()
            if not line:
                break
            if self.is_newEntry(line.strip()) is not True:
                print()
        f.close()